In [38]:
import webbrowser
import os
import shutil
import requests
import re
new = 2


#open an HTML file on my own (Windows) computer
def open_html(relpath):
    url = 'file:///Users/aleks/Documents/study/phd/deeptechNikolaev/sketchfabAutoDownload/' + relpath
    # webbrowser.open(url)
    webbrowser.open(url,new=new)

# Fetching model uids

In [30]:
collection_id = '41f58a2863d9493b9ef9b3ffc95954eb'

response = requests.get('https://api.sketchfab.com/v3/collections/'+collection_id+'/models')
assert response.status_code==200
# res = response.json()['results'][0]
uids_collection = []
for res in response.json()['results']:
    uids_collection.append(res['uid'])
uids_collection
n_models = 1

# Preparing `iframe`s for `index.html`

In [31]:
with open('index_empty.html') as f:
    data = f.readlines()

In [32]:
template = [
'            <div class="row ">\n',
'                <div class="d-flex flex-column col-md-8 mb-3">\n',
'                    <iframe id="api-frame" width="auto" height="auto" allowfullscreen="true" mozallowfullscreen="true"\n',
'                        webkitallowfullscreen="true"\n',
'                        class="mb-3 rounded shadow-lg embed-responsive embed-responsive-4by3 border border-info"\n',
'                        loading="lazy"></iframe>\n',
'                </div>\n',
'            </div>\n'
]
def add_after_subs(string, substring, i):
    s_pos = string.find(substring)
    e_pos = s_pos + len(substring)
    string = string[:e_pos] + str(i) + string[e_pos:]
    return string
def alter_template(i):
    if i == 0:
        return template
    
    template_i = template.copy()
    # <div class="row ">
    template_i[0]=add_after_subs(template_i[0], 'row', i)
    # <iframe id="api-frame1" ...
    template_i[2]=add_after_subs(template_i[2], 'api-frame', i)
    return template_i

In [33]:
iframes_html = []
for i, uid in enumerate(uids_collection[-n_models:]):
    iframes_html += alter_template(i)

In [34]:
tmp=data[:14] + iframes_html + data[14:]

In [35]:
with open('index_tst.html', 'w') as the_file:
    for l in tmp:
        the_file.write(l)

In [36]:
js_file_path = 'init_model.js'
with open(js_file_path, 'r') as file:
    # read a list of lines into data
    data = file.readlines()
#find uids row
uids_row_idx = -1
for i, l in enumerate(data):
    if 'var uids = ' in l:
        uids_row_idx = i
        break
assert uids_row_idx > 0
#alter it
data[uids_row_idx] = 'var uids = ' + str(uids_collection[-n_models:]) + ';\n'

with open('init_model.js', 'w') as file:
    file.writelines(data)

# Bonus: gifs

In [51]:
# filenames = 
p = re.compile('.*tst.*')
filenames = [os.path.join('/Users/aleks/Downloads/',s) for s in os.listdir('/Users/aleks/Downloads/') if p.match(s)]
def key(x):
    try: 
        return int(x.split('-')[-1].split('.')[0])
    except ValueError:
        return 0
filenames.sort(key=lambda x: )

ValueError: invalid literal for int() with base 10: '/Users/aleks/Downloads/model_7e8a61dd67b341e987e88299af27fe57_tst'

In [50]:
filenames

['/Users/aleks/Downloads/model_7e8a61dd67b341e987e88299af27fe57_tst-10.jpg',
 '/Users/aleks/Downloads/model_7e8a61dd67b341e987e88299af27fe57_tst-11.jpg',
 '/Users/aleks/Downloads/model_7e8a61dd67b341e987e88299af27fe57_tst-12.jpg',
 '/Users/aleks/Downloads/model_7e8a61dd67b341e987e88299af27fe57_tst-13.jpg',
 '/Users/aleks/Downloads/model_7e8a61dd67b341e987e88299af27fe57_tst-14.jpg',
 '/Users/aleks/Downloads/model_7e8a61dd67b341e987e88299af27fe57_tst-15.jpg',
 '/Users/aleks/Downloads/model_7e8a61dd67b341e987e88299af27fe57_tst-16.jpg',
 '/Users/aleks/Downloads/model_7e8a61dd67b341e987e88299af27fe57_tst-17.jpg',
 '/Users/aleks/Downloads/model_7e8a61dd67b341e987e88299af27fe57_tst-18.jpg',
 '/Users/aleks/Downloads/model_7e8a61dd67b341e987e88299af27fe57_tst-19.jpg',
 '/Users/aleks/Downloads/model_7e8a61dd67b341e987e88299af27fe57_tst-2.jpg',
 '/Users/aleks/Downloads/model_7e8a61dd67b341e987e88299af27fe57_tst-20.jpg',
 '/Users/aleks/Downloads/model_7e8a61dd67b341e987e88299af27fe57_tst-21.jpg',


In [49]:
import imageio
with imageio.get_writer('/Users/aleks/Downloads/movie.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

/var/folders/x3/2q_gzsqs643dbh4q2k13s0z80000gn/T/ipykernel_2585/778663958.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
